In [ ]:
import json, pandas

In [ ]:
BROKER_DICT = None
with open('broker.json', "r") as broker_json_file:
    BROKER_DICT = json.load(broker_json_file)

SYMBOLS = ["vale3","petr4", "bpac11", "ceab3", "abev3", "b3sa3", "embr3", "ggbr4", "tots3"]

def broker_ranking_by_ticker(trades): 
    pnl={}
    for i,trade in trades.iterrows():
        buyer = trade['buyer']
        seller = trade['seller']
        qty = trade['qty']
        price = trade['price']
        volume = price * qty
        agressor_buyer = 1 if trade['agressor'] == 'A' else 0
        agressor_seller = 1 if trade['agressor'] == 'V' else 0
        buyer_stats_from_dict = pnl.get(buyer)
        if buyer_stats_from_dict is None:
            stats = {}
            stats['net'] = volume
            stats['qty'] = qty
            stats['agressor'] = agressor_buyer
            pnl[buyer] = stats
        else:
            buyer_stats_from_dict['net'] = buyer_stats_from_dict['net'] + volume
            buyer_stats_from_dict['qty'] = buyer_stats_from_dict['qty'] + qty
            buyer_stats_from_dict['agressor'] = buyer_stats_from_dict['agressor'] + agressor_buyer
            pnl[buyer] = buyer_stats_from_dict

        seller_stats_from_dict = pnl.get(seller)
        if seller_stats_from_dict is None:
            stats = {}
            stats['net'] = -volume
            stats['qty'] = qty
            stats['agressor'] = agressor_seller
            pnl[seller] = stats
        else:
            seller_stats_from_dict['net'] = seller_stats_from_dict['net'] - volume
            seller_stats_from_dict['qty'] = seller_stats_from_dict['qty'] + qty
            seller_stats_from_dict['agressor'] = seller_stats_from_dict['agressor'] + agressor_seller
            pnl[seller] = seller_stats_from_dict

    statistics_array = []
    for idx in pnl:
        broker_name = [x for x in BROKER_DICT if x['playerMarketCode'] == str(idx)][1]['playerNameFull']
        statistics_array.append([idx, broker_name, pnl[idx]['net'], pnl[idx]['qty'], pnl[idx]['agressor']])
    statistics = pandas.DataFrame(statistics_array, columns=['BROKER_ID', 'BROKER_NAME', 'NET', 'QTY', 'AGRESSOR'])
    return statistics

In [ ]:
trades = pandas.read_csv('20210420_bbt.log.csv',sep=';')
for symbol in SYMBOLS:
    print("\n\n")
    print("============================================================================")
    print("===================================", symbol.upper(),"==================================")
    print("============================================================================")
    stats = broker_ranking_by_ticker(trades[trades['symbol'] == symbol.upper()])
    print(stats.sort_values('QTY',ascending=False))

In [ ]:
trades.head()

In [ ]:
trades[trades.symbol=='CEAB3']['qty'].head()